In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr


In [2]:
city = 'MILANO'
prediction_name = 'UHI_ANN_predictions_acc_77_feb_14_100mil'#'UHI_ANN_sim_veg2'
#prediction_name = 'UHI_ANN_predictions_acc_84_DUSAF_new'
base_folder = f'predictions'
data_folder = "data"

file_name = f'{city}_{prediction_name}.csv'
prob = pd.read_csv(f'{base_folder}/{file_name}', index_col=0)

# get raster parameters
#landcover_path = f"{data_folder}/landcover/DUSAF_2021_milano_mapped.tif"
#landcover_path = f"{data_folder}/MILANO_landcover.tif"
landcover_path = f"{data_folder}/landcover/DUSAF_2021_MCM_mapped.tif"
#landcover_path = '/home/user/ODC_harmonia/Landsat/Milan/data/clip/LC08_L2SP_194028_20220810_20220818_02_T1/LC08_L2SP_194028_20220810_20220818_02_T1_LST.TIF'
with rio.open(landcover_path, driver="GTiff") as base_raster:
    transform = base_raster.transform
    init_x = transform[2]
    init_y = transform[5]
    step_x = transform[0]
    step_y = transform[4]
    raster_w = base_raster.width
    raster_h = base_raster.height
    profile = base_raster.profile
    
transform

Affine(5.0, 0.0, 477051.11479999963,
       0.0, -5.0, 5054450.992000001)

In [3]:
prob

,probability,x,y
0,0.5215,495341.1148,5054280.992
1,0.7219,495346.1148,5054280.992
2,0.7218,495351.1148,5054280.992
3,0.5214,495321.1148,5054275.992
4,0.5214,495326.1148,5054275.992
...,...,...,...
61865261,0.7583,537921.1148,5001000.992
61865262,0.7586,537926.1148,5001000.992
61865263,0.7589,537931.1148,5001000.992
61865264,0.5870,537936.1148,5001000.992


In [4]:
da = prob.set_index(['y', 'x']).to_xarray()
da['y'] = da['y'].astype(int)
da['x'] = da['x'].astype(int)
da = da.astype('float64')
da

<xarray.Dataset>
Dimensions:      (y: 9278, x: 13198)
Coordinates:
  * y            (y) int64 5001000 5001005 5001010 ... 5054270 5054275 5054280
  * x            (x) int64 477071 477076 477081 477086 ... 543046 543051 543056
Data variables:
    probability  (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [5]:
# Generate the full grid with the base raster parameters
full_grid_ds = xr.Dataset(
    coords=dict(
        x=np.arange(init_x, init_x + (step_x * raster_w), step_x, dtype=int),
        y=np.arange(init_y, init_y + (step_y * raster_h), step_y, dtype=int),
    )
)
full_grid_ds

<xarray.Dataset>
Dimensions:  (x: 13202, y: 10691)
Coordinates:
  * x        (x) int64 477051 477056 477061 477066 ... 543046 543051 543056
  * y        (y) int64 5054450 5054445 5054440 ... 5001010 5001005 5001000
Data variables:
    *empty*

In [6]:
#Combine the probability points with the grid for filling the complete raster matrix
arr1 = da
arr2 = full_grid_ds

combined_arr = arr2.combine_first(arr1)
combined_arr = combined_arr.fillna(9999)
combined_arr = combined_arr.astype('float32')
combined_arr

<xarray.Dataset>
Dimensions:      (y: 10691, x: 13202)
Coordinates:
  * x            (x) int64 477051 477056 477061 477066 ... 543046 543051 543056
  * y            (y) int64 5001000 5001005 5001010 ... 5054440 5054445 5054450
Data variables:
    probability  (y, x) float32 9.999e+03 9.999e+03 ... 9.999e+03 9.999e+03

In [7]:
#copy raster profile from the landcover
prediction_profile = profile.copy()
prediction_profile['dtype'] = 'float32'
prediction_profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 9999.0, 'width': 13202, 'height': 10691, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(5.0, 0.0, 477051.11479999963,
       0.0, -5.0, 5054450.992000001), 'tiled': False, 'interleave': 'band'}

In [10]:
#print(array_to_write.shape, (prediction_profile['height'], prediction_profile['width']))


In [9]:
raster_path = f'rasters/{prediction_name}.tif'
# the array is initially flipped in x and y axis.
array_to_write = np.flip(np.flip(combined_arr.probability.to_numpy()), axis=1)

with rio.open(raster_path, 'w', **prediction_profile) as dest:
    dest.write(array_to_write, 1)